# **ETL del dataset:** *user_reviews*

#### **Importación de librerías** ####
---

*Para este cuaderno usaremos las siguientes librerías: **pandas**, **gzip**, **ast**, **pyarrow** y **textblob**.*

In [ ]:
import pandas as pd
import gzip
import ast
import pyarrow as pa
import pyarrow.parquet as pq
from textblob import TextBlob as tb

#### **Extracción de los datos** ####
---

*Procedemos a abrir nuestro dataset y almacenarlo en un dataframe para visualizarlo y manejarlo en pandas.*

1. *Encontramos un error en el formato del dataset que no permite abrirlo con la **forma tradicional**.*
2. *Creamos **data_list** que almacenará cada una de las líneas de nuestro archivo.*
3. *Iniciamos una **iteración** en el archivo, esto leerá línea por línea el archivo.*
4. *Usamos la función **literal_eval** para convertir cada línea en una estructura de datos Python.*
5. *Creamos el dataset con base a la lista.*

In [2]:
ruta = 'datasets/user_reviews.json.gz'

In [3]:
data_list = []

for line in gzip.open(ruta):
    data_list.append(ast.literal_eval(line.decode("utf-8")))

In [4]:
dfUsersReviews = pd.DataFrame(data_list)
dfUsersReviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


**Transformación columna reviews**

1. *Usamos el metodo **explode** para expandir la columna reviews, descomponiendo cada lista en filas separadas.*
2. *Este resultado lo almacenamos en la variable **colReviews***
3. *Extraemos los datos anidados en **colReviews** y los normalizamos con el método **json_normalize** y usamos **dropna** para eliminar cuaquier dato nulo que contenga valores nulos en la columna review.*

In [5]:
colReviews = dfUsersReviews.explode("reviews")
colReviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [6]:
dfUsersReviews = pd.json_normalize(colReviews["reviews"].dropna())
dfUsersReviews

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...
59300,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
59301,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
59302,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59303,,Posted July 20.,,730,No ratings yet,True,:D


### **Análisis Exploratorio Inicial** ###
---

*Vamos a realizar una **exploración de los datos** (EDA) antes de hacer transformación de los mismos, esto con el fin de **conocer las columnas** y determinar que campos son valiosos para nuestro análisis y así tener claro que pasos realizaremos en el ETL.*

**Nota:**: *Posterior al ETL realizaremos también un [EDA](04_EDA.ipynb)
 enfocado en los datos y una descripción estadística.*

#### **Revisión de la extracción de los datos** ####

*Revisamos que los datos se cargaron adecuadamente en nuestro dataframe:*

1. *Usamos **head** para previsualizar los primeros registros del Dataframe.*
2. *Podemos observar qué se cargaron adecuadamente los datos, pero vemos datos **Sin registro** en las columnas last_edited y funny.*
3. *Podemos observar qué en la columna posted tenemos fechas con la palabra **Posted**.*

In [ ]:
dfUsersReviews.head(3)

*Revisemos si este patrón se repite en todos los datos:*

1. *Usamos **tail** para previsualizar los últimos registros del Dataframe.*
2. *Observamos que los últimos registros del Dataframe si tienen **datos "válidos"** en la columna funny.*
3. *A vista general podemos observar que en nuestro Dataframe existen **datos vacíos**, pero esto lo revisaremos más adelante.*

In [8]:
dfUsersReviews.tail(3)

,funny,posted,last_edited,item_id,helpful,recommend,review
59302,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59303,,Posted July 20.,,730,No ratings yet,True,:D
59304,,Posted July 2.,,440,No ratings yet,True,so much fun :D


*Revisamos la forma del dataframe:*

1. *Usamos **shape** obtener el número de filas y columnas del dataframe.*
2. *Nuestro dataframe tiene **59305 filas y 7 columnas**.*

In [9]:
dfUsersReviews.shape

(59305, 7)

#### **Exploración de las columnas** ####

*Vamos a revisar qué columnas existen en el dataframe y de qué tipo son:*

1. *Usamos el método **dtypes** para listar las columnas y sus tipos.*
2. *Observamos que 6 columnas contienen cadenas de caracteres identificados como **object** y 1 columna contiene datos **booleanos**.*

In [10]:
dfUsersReviews.dtypes

funny          object
posted         object
last_edited    object
item_id        object
helpful        object
recommend        bool
review         object
dtype: object

*Revisemos qué tipos de datos almacena cada columna a detalle:*

1. *Hacemos una **iteración** que nos permite recorrer los valores de cada columna para encontrar sus tipos.*
2. *Observamos que todas las columnas tienen definido **un solo tipo** de dato.*

In [11]:
tipoDatos = {"Columna":[], "Tipos":[]}
for c in dfUsersReviews.columns:
    tipoDatos["Columna"].append(c)
    tipoDatos["Tipos"].append(dfUsersReviews[c].apply(type).unique())
    
dfTypes = pd.DataFrame(tipoDatos)
dfTypes

,Columna,Tipos
0,funny,[<class 'str'>]
1,posted,[<class 'str'>]
2,last_edited,[<class 'str'>]
3,item_id,[<class 'str'>]
4,helpful,[<class 'str'>]
5,recommend,[<class 'bool'>]
6,review,[<class 'str'>]


*Revisemos un poco más:*

1. *Usamos **describe()** para obtener un resumen de estas columnas.*
2. *Por ahora no dicen mucho sobre nuestros datos ya que debemos **cambiar algunos tipos**.*

In [12]:
dfUsersReviews.describe()

,funny,posted,last_edited,item_id,helpful,recommend,review
count,59305,59305,59305,59305,59305,59305,59305
unique,186,1906,1015,3682,1344,2,55313
top,,"Posted June 21, 2014.",,730,No ratings yet,True,good game
freq,51154,225,53165,3759,30168,52473,100


*Revisemos los valores alojados en cada columna para saber cuantos son validos, es decir no nulos:*

1. *Usamos **info()** para obtener un conteo de los valores válidos por columnas*
2. *Por ahora podemos observar que no tenemos valores **nulos**. Pero en un análisis anterior observabamos que habían valores vacíos.*

In [13]:
dfUsersReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   funny        59305 non-null  object
 1   posted       59305 non-null  object
 2   last_edited  59305 non-null  object
 3   item_id      59305 non-null  object
 4   helpful      59305 non-null  object
 5   recommend    59305 non-null  bool  
 6   review       59305 non-null  object
dtypes: bool(1), object(6)
memory usage: 2.8+ MB


*Descripción de las variables del dataframe:*

1. *Creamos una tabla para **describir y comparar las columnas**.*

| Variable    | Definición                                              | Tipo de Dato | Tipo de Variable    |
|:------------|:--------------------------------------------------------|:------------:|:--------------------|
| Funny       | Cantidad de personas que consideran la reseña divertida | String       | Numérica(discreta)  |
| Posted      | Fecha de publicación de la reseña                       | String       | Categórica(ordinal) |
| Last Edited | Ultima edición de la reseña                             | String       | Categórica(ordinal) |
| Item Id     | Identificador de la app                                 | Integer      | Numérica(ordinal)   |
| Helpful     | Cantidad de personas que consideran la reseña de ayuda  | String       | Numérica(discreta)  |
| Recommend   | Si se recomienda o no esta app                          | Bool         | Catégorica(nominal) |
| Review      | Comentarios de la reseña                                | String       | Catégorica(nominal) |

2. *Revisamos las especificaciones de las funciones que requiere nuestro **MVP** para saber que columnas serán importantes para las funcionalidades de la API.*
3. *Observamos que existen columnas como **Funny**, **Posted**, **LastEdited**, **HelpFul** que no serán apropiadas para los objetivos de este análisis.*
4. *Se observa que algunas columnas necesitan transformaciones en sus datos.*

**Nota:** *En algunos visualizadores de Markdown no es posible apreciar la tabla, sugiero revisarla en VSC.*

### **Transformación de los datos** ###
---

*A partir de este momento realizaremos una limpieza de los datos teniendo en cuenta nuestro análisis previo.*

* **Nota:** *Posterior al ETL realizaremos también un [EDA](04_EDA.ipynb)
 enfocado en los datos y una descripción estadística.*

#### Tranformación por columnas ####

*A continuación vamos a hacer limpieza por columnas, pero antes haremos algunos cambios:*

1. **Reorganizamos las columnas** *, con el fin de mejorar la lectura y comparación de los datos y columnas*.
2. **Cambiamos los nombres** *de las columnas.*

In [14]:
dfUsersReviews.columns

Index(['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend',
       'review'],
      dtype='object')

In [15]:
dfUsersReviews= dfUsersReviews[["item_id", "posted", "last_edited","recommend", "helpful", "funny", "review"]]

In [16]:
dfUsersReviews.columns = ['IdApp', 'PostedDate', "LastEdited", "IsRecommend", "IsHelpful", "IsFunny", "Comments"]

**Columna IdApp**

1. *Cambiamos el tipo de dato a **int**.*
2. *Reorganizamos los datos por **id**.*

In [17]:
dfUsersReviews["IdApp"] = dfUsersReviews["IdApp"].astype(int)

In [18]:
dfUsersReviews = dfUsersReviews.sort_values("IdApp")
dfUsersReviews = dfUsersReviews.reset_index(drop=True)
dfUsersReviews.head(3)

,IdApp,PostedDate,LastEdited,IsRecommend,IsHelpful,IsFunny,Comments
0,10,"Posted April 7, 2012.",,True,No ratings yet,,Good Game :D
1,10,"Posted March 19, 2014.",,True,0 of 2 people (0%) found this review helpful,,One of the best childhood games i have ever pl...
2,10,"Posted August 16, 2013.",,True,No ratings yet,,jueguenlooooooo


**Columna PostedDate**

1. *Observamos que esta columna tiene las fechas de lanzamiento de cada aplicación en **formato MES-DIA-AÑO**, aunque existen algunas excepciones como la palabra Posted*.
2. *Para efectos del análisis se decide **dividir esta columna en dos**, una columna con el año de posteo y otra columna con el mes de posteo*.

In [19]:
for i, e in enumerate(dfUsersReviews["PostedDate"]):
    dfUsersReviews.loc[i, "PostedMonth"] = e[7:]
    dfUsersReviews.loc[i, "PostedYear"] = e[7:]

3. *Aplicamos un **iterador** que enumere la columna PostedMonth y divida los datos en dos, con el fin de quedarnos con la primera parte antes del espacio.*

In [20]:
for i, e in enumerate(dfUsersReviews["PostedMonth"]):
    dfUsersReviews.loc[i, "PostedMonth"] = e.split(" ", 1)[0]

4. *Aplicamos un **iterador** que enumere la columna PostedYear y se quede con los ultimos 5 caracteres.*

In [21]:
for i, e in enumerate(dfUsersReviews["PostedYear"]):
    dfUsersReviews.loc[i, "PostedYear"] = e[-5:]

5. *Aplicamos un **iterador** que enumere la columna PostedYear y elimine el último caracter.*

In [22]:
for i, e in enumerate(dfUsersReviews["PostedYear"]):
    dfUsersReviews.loc[i, "PostedYear"] = e[:-1]

6. *Aplicamos un **iterador** que enumere la columna PostedYear y cuando encuentre un dato que no inicie con un digito, lo cambie por Sin Dato.*

In [23]:
for i, e in enumerate(dfUsersReviews["PostedYear"]):
    if e.isdigit() == False:
        dfUsersReviews.loc[i, "PostedYear"] = "Sin Dato"

7. *Eliminamos la columna **PostedDate**.*

In [24]:
dfUsersReviews = dfUsersReviews.drop(columns= "PostedDate")
dfUsersReviews.head(3)

,IdApp,LastEdited,IsRecommend,IsHelpful,IsFunny,Comments,PostedMonth,PostedYear
0,10,,True,No ratings yet,,Good Game :D,April,2012
1,10,,True,0 of 2 people (0%) found this review helpful,,One of the best childhood games i have ever pl...,March,2014
2,10,,True,No ratings yet,,jueguenlooooooo,August,2013


**Columna LastEdited**

1. *Realizamos un **conteo** de los valores unicos que tiene esta columna.*
2. *Observamos que **53156** datos son vacíos de un total de **59305** datos*.
3. *Por lo tanto **eliminamos esta columna**, ya que no nos aporta mucha información*.

In [25]:
conteoLastEdit = dfUsersReviews["LastEdited"].value_counts()
conteoLastEdit = pd.DataFrame(conteoLastEdit)
conteoLastEdit

,count
LastEdited,
,53165
"Last edited November 25, 2013.",99
"Last edited October 17, 2015.",19
"Last edited June 6, 2015.",18
"Last edited July 25, 2015.",17
...,...
"Last edited February 28, 2014.",1
"Last edited September 16, 2015.",1
"Last edited February 11, 2014.",1


In [26]:
dfUsersReviews.drop(columns="LastEdited", inplace=True)

In [27]:
dfUsersReviews

,IdApp,IsRecommend,IsHelpful,IsFunny,Comments,PostedMonth,PostedYear
0,10,True,No ratings yet,,Good Game :D,April,2012
1,10,True,0 of 2 people (0%) found this review helpful,,One of the best childhood games i have ever pl...,March,2014
2,10,True,No ratings yet,,jueguenlooooooo,August,2013
3,10,True,1 of 1 people (100%) found this review helpful,,Counter-Strike makes you work to enjoy the con...,December,2015
4,10,True,1 of 1 people (100%) found this review helpful,,Just coz its old doesn't mean its good.ProsSki...,April,2014
...,...,...,...,...,...,...,...
59300,521430,True,3 of 5 people (60%) found this review helpful,2 people found this review funny,"A neat platformer, that has a nice geometric l...",August,Sin Dato
59301,521570,True,No ratings yet,,kill me.,August,Sin Dato
59302,521570,True,No ratings yet,1 person found this review funny,6-5 =/,August,Sin Dato
59303,521990,True,No ratings yet,,This is one of the nicest looking shoot'em ups...,September,Sin Dato


**Columna IsRecommend**

1. *Vamos a **verificar** que valores distintos tenemos.*
2. *No se observan anomalias en esta tabla.*

In [28]:
conteoIsRec = dfUsersReviews["IsRecommend"].value_counts()
conteoIsRec = pd.DataFrame(conteoIsRec)
conteoIsRec

,count
IsRecommend,
True,52473
False,6832


**Columna IsHelpful**

1. *Vamos a **verificar** que valores distintos tenemos.*
2. *Podríamos transformar la columna para establecer el **% de ayuda de cada review**, sin embargo para los efectos de este análisis esta columna no es significativa.*

In [29]:
conteoIsHelp = dfUsersReviews["IsHelpful"].value_counts()
conteoIsHelp = pd.DataFrame(conteoIsHelp)
conteoIsHelp

,count
IsHelpful,
No ratings yet,30168
1 of 1 people (100%) found this review helpful,6730
0 of 1 people (0%) found this review helpful,4024
1 of 2 people (50%) found this review helpful,2493
2 of 2 people (100%) found this review helpful,1872
...,...
116 of 139 people (83%) found this review helpful,1
423 of 598 people (71%) found this review helpful,1
35 of 39 people (90%) found this review helpful,1


In [30]:
dfUsersReviews.drop(columns="IsHelpful", inplace=True)

**Columna IsFunny**

1. *Vamos a **verificar** que valores distintos tenemos.*
2. *Podríamos transformar la columna para establecer el **% de divertido**, sin embargo para los efectos de este análisis esta columna no es significativa ya que nuestro objetivo no es evaluar las calificaciones de las reviews, sino las calificaciones de las apps.*

In [32]:
conteoIsFun = dfUsersReviews["IsFunny"].value_counts()
conteoIsFun = pd.DataFrame(conteoIsFun)
conteoIsFun

,count
IsFunny,
,51154
1 person found this review funny,5180
2 people found this review funny,1232
3 people found this review funny,491
4 people found this review funny,267
...,...
85 people found this review funny,1
296 people found this review funny,1
279 people found this review funny,1


In [33]:
dfUsersReviews.drop(columns="IsFunny", inplace=True)

**Columna Comments**

1. *Vamos a **verificar** que valores distintos tenemos o que se repiten*
2. *Observamos que uno de los comentarios que más se repiten es good game, esto nos servirá más adelante.*

In [35]:
conteoCom = dfUsersReviews["Comments"].value_counts()
conteoCom = pd.DataFrame(conteoCom)
conteoCom

,count
Comments,
good game,100
good,98
great game,83
10/10,76
Great game,50
...,...
"an amazing game, and great physixs",1
"This game really did surprise me with its gameplay. Usually WWII games are a bore to me, yet this was not the case for Sniper Elite v2. If you love seeing slow-mo cinematics of your enemies' heads exploding into pieces, this is a great game for you.""Fallout V.A.T.S. on steroids""",1
literally the worst ♥♥♥♥ i have ever played,1


### **Análisis de Sentimientos** ###
---

*Vamos a aplicar a la columna **Comments** que contiene las reseñas de cada una de las apps, un análisis con la ayuda de herramientas de **Procesamiento de Lenguaje Natural (NLP)**. Para llevar a cabo este análisis usaremos TextBlob, una herramienta para procesar datos de texto, construida sobre **Natural Language Toolkit (NLTK)***.

**Nota:** *También construiremos un nuevo dataframe con los resultados de este análisis.*

|        Ejemplo       | Sentimiento | Categoría |
|:--------------------:|:-----------:|:---------:|
| "Bad :c"             | Malo        | 0         |
| "Nothing to say"     | Neutral     | 1         |
| "The best game ever" | Positivo    | 2         |


*Vamos a crear una función que podamos usar cuando lleguen más datos a nuestro dataframe*

1. *Esta función se llamará **detector_sentimiento** y recibirá como parametro una cadena de caracteres y dos parametros opcionales **(algo así como hiperparametros en un modelo de Machine Learning)** que permiten ajustar el grado de polaridad.*
2. *Si la reseña como parametro que recibe es **nula** retornará 1.*

In [50]:
def detector_sentimiento(reseña: str, polPos = 0.05, polNeg = -0.05):
    """
    Clasificar el sentimiento en tres categorías.

    Esta función procesa una cadena de caracteres para clasificarla en tres tipos de sentimientos:
        - Negativo (0)
        - Neutral (1)
        - Positivo (2)

    Parámetros:
    reseña = Este input debe ser una cadena de caracteres y lo más normalizado posible, sin anidamientos.
    
    polPos = Este input establece el valor de polaridad positiva para considerar que un sentimiento es positivo
            Si el valor no se pasa al llamar la función, su valor predeterminado será 0.05.
            
    polPos = Este input establece el valor de polaridad positiva para considerar que un sentimiento es negativo
            Si el valor no se pasa al llamar la función, su valor predeterminado será -0.05.
    

    Valor de retorno:
    sentimiento: Retorna un valor entero entre 0 y 2, dependiendo los resultados del análisis.

    Nota: En caso de encontrar un valor nulo, esta retornará un 1.
    """
    if reseña == None:
        return 1
    else:
        #Instanciamos la clase Textblob y le pasamos por parametro la cadena de caracteres.
        detect = tb(reseña)
        # Obtenemos la polaridad identficada por Textblob.
        polar = detect.sentiment.polarity
        
        #Definimos los umbrales que manejará
        if polar >polPos:
            return 2
        elif polar < polNeg:
            return 0
        else:
            return 1

*Creamos una copia de nuestro dataframe original que va acontener los análisis de sentimiento*

1. *Este dataframe se llamará **dfFeelings**.*

In [77]:
dfFeelings = dfUsersReviews[["IdApp","Comments"]].copy()
dfFeelings.head(3)

,IdApp,Comments
0,10,Good Game :D
1,10,One of the best childhood games i have ever pl...
2,10,jueguenlooooooo


*Ahora vamos a entrenar nuestro modelo con el fin ajustarlo adecuadamente a los comentarios originales.*

1. *Usamos la polaridad predefinida en la función: PolPos = 0.05 y PolNeg= -0.05*
2. *Usamos los siguientes parametros PolPos = 0.01 y PolNeg= -0.01*
3. *Usamos los siguientes parametros PolPos = 0.1 y PolNeg= -0.1*
4. *Usamos los siguientes parametros PolPos = 0.5 y PolNeg= -0.5*
5. *Usamos los siguientes parametros PolPos = 0.75 y PolNeg= -0.75*

In [78]:
dfFeelings["SenAn1"] = dfFeelings["Comments"].apply(detector_sentimiento)
dfFeelings["SenAn2"] = dfFeelings["Comments"].apply(detector_sentimiento, polPos=0.01, polNeg=-0.01)
dfFeelings["SenAn3"] = dfFeelings["Comments"].apply(detector_sentimiento, polPos=0.1, polNeg=-0.1)
dfFeelings["SenAn4"] = dfFeelings["Comments"].apply(detector_sentimiento, polPos=0.5, polNeg=-0.5)
dfFeelings["SenAn5"] = dfFeelings["Comments"].apply(detector_sentimiento, polPos=0.75, polNeg=-0.75)

*Ahora revisamos los resultados*

1. *Observamos que los analisis que más se ajustan a los comentarios son: Analisis 1 y Analisis 2*
2. *Sin embargo el análisis 1 tiene mayor ajuste en la celda 35, mientras que el analisis 2 en vez de categorizar un sentimiento como positivo lo categoriza como negativo.*

In [79]:
dfFeelings.loc[30:40]

,IdApp,Comments,SenAn1,SenAn2,SenAn3,SenAn4,SenAn5
30,10,Best server Ffa classic [DG-Zone]Classic Pub S...,2,2,2,2,1
31,10,"Counter Strike is a classic game. Fun, addicti...",2,2,1,1,1
32,10,Siq game,0,0,0,1,1
33,10,2012-2014: awsome classic game changing shoot...,0,0,1,1,1
34,10,Esse jogo é muito bom recomendo e ganha do cs ...,1,1,1,1,1
35,10,Cool game,1,0,1,1,1
36,10,"1,6 > go fk go.",1,1,1,1,1
37,10,Que decir de este gran juego?? Comence a jugar...,1,1,1,1,1
38,10,2gud4u,1,1,1,1,1
39,10,clickin heads since i was 12,1,1,1,1,1


In [80]:
dfFeelings.drop(columns=["SenAn3", "SenAn4","SenAn5","SenAn2"], inplace=True)

In [81]:
dfFeelings.rename(columns={"SenAn1":"SentimentalAnalysis"})

,IdApp,Comments,SentimentalAnalysis
0,10,Good Game :D,2
1,10,One of the best childhood games i have ever pl...,2
2,10,jueguenlooooooo,1
3,10,Counter-Strike makes you work to enjoy the con...,2
4,10,Just coz its old doesn't mean its good.ProsSki...,2
...,...,...,...
59300,521430,"A neat platformer, that has a nice geometric l...",1
59301,521570,kill me.,1
59302,521570,6-5 =/,0
59303,521990,This is one of the nicest looking shoot'em ups...,2


### **Análisis Exploratorio Final** ###
---

*Aquí hacemos una revisión de los datos después de su transformación, con el objetivo de revisar si quedaron datos pendientes por transformar*.

**Nota:** *Posterior a todas las transformaciones se realiza un [EDA](04_EDA.ipynb)
 final.*

*Obtenemos una descripción de las columnas, sus tipos de datos y sus valores no nulos:*

1. *El total de registros es de **59305**, y no se encuentran valores nulos.*

In [82]:
dfUsersReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   IdApp        59305 non-null  int32 
 1   IsRecommend  59305 non-null  bool  
 2   Comments     59305 non-null  object
 3   PostedMonth  59305 non-null  object
 4   PostedYear   59305 non-null  object
dtypes: bool(1), int32(1), object(3)
memory usage: 1.6+ MB


### **Exportación de datos a Parquet (Load)** ###
---

*Una vez hemos revisado nuestro dataframe, lo podemos exportar a un archivo **parquet** para guardar todos los cambios realizados*.


**Nota:** *Los archivos se encuentran en la carpeta dataout.*

In [140]:
# dfUsersReviews.to_csv("datasets/out_users_reviews.csv", index=False, encoding="utf-8")

In [83]:
usersRevTab = pa.Table.from_pandas(dfUsersReviews)
dir = "dataout/out_users_reviews.parquet"
pq.write_table(usersRevTab,dir)

In [84]:
feelingsTab = pa.Table.from_pandas(dfFeelings)
dir1 = "dataout/out_feelings.parquet"
pq.write_table(feelingsTab,dir1)